<a href="https://colab.research.google.com/github/sidh26/NLP-assignments/blob/master/classwork/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv")
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
df.fillna(' ', inplace=True)
df['text'] = df['Title'].astype(str) + ' ' + df['Review Text'].astype(str)
df = pd.concat([df, pd.get_dummies(df[['Division Name', 'Department Name', 'Class Name']], drop_first=True)], axis=1)
df.drop(['Unnamed: 0', 'Division Name', 'Department Name', 'Class Name', 'Title', 'Review Text'],axis=1, inplace=True)
df.head()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count,text,Division Name_General,Division Name_General Petite,Division Name_Initmates,Department Name_Bottoms,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Blouses,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,767,33,4,1,0,Absolutely wonderful - silky and sexy and co...,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1080,34,5,1,4,Love this dress! it's sooo pretty. i happe...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1077,60,3,0,0,Some major design flaws I had such high hopes ...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049,50,5,1,0,"My favorite buy! I love, love, love this jumps...",0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,847,47,5,1,6,Flattering shirt This shirt is very flattering...,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import pandas as pd
import re
import numpy as np
from nltk import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from itertools import combinations

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
def preprocess(doc):
    sentences = []
    doc = doc['text'].lower()
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(doc)
    t = [w for w in word_tokens if not w in stop_words]
    t = ' '.join(t)
    t = re.sub(r'[^A-Za-z\s]', ' ', t)
    return t

In [0]:
df['text'] = df.apply(preprocess, axis=1)

In [8]:
df.head()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count,text,Division Name_General,Division Name_General Petite,Division Name_Initmates,Department Name_Bottoms,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Blouses,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,767,33,4,1,0,absolutely wonderful silky sexy comfortable,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1080,34,5,1,4,love dress s sooo pretty happened find st...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1077,60,3,0,0,major design flaws high hopes dress really wan...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049,50,5,1,0,favorite buy love love love jumpsuit ...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,847,47,5,1,6,flattering shirt shirt flattering due adjustab...,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split as tts

In [0]:
# steps=[('Preprocess', Preprocess()),('Countvectorizer', CountVectorizer()),('rf', LogisticRegression())]
# pp=Pipeline(steps)

In [0]:
steps=[('Countvectorizer', CountVectorizer()),('rf', RandomForestClassifier())]
pp=Pipeline(steps)

In [19]:
from sklearn.model_selection import GridSearchCV
params = {'rf__max_depth': [10, 20],
 'rf__min_samples_leaf': [1, 2],
 'rf__min_samples_split': [2, 5],
'rf__n_estimators': range(10,20,50)
}
grid = GridSearchCV(pp, param_grid=params, cv=5,verbose=1)
x_train,x_test,y_train,y_test=tts(df.iloc[:,5:],df['Recommended IND'])
grid.fit(x_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Number of labels=14091 does not match number of samples=30

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Number of labels=14091 does not match number of samples=30

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Number of labels=14091 does not match number of samples=30

  FitFailedWarning)
/usr

ValueError: ignored

In [0]:
grid.predict(y_test)

In [0]:
grid.score(x_test,y_test)

In [0]:
class Preprocess():
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def clean(self, doc):
        sentences = []
        doc = doc['text'].lower()
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(doc)
        t = [w for w in word_tokens if not w in stop_words]
        t = ' '.join(t)
        t = re.sub(r'[^A-Za-z\s]', ' ', t)
        return t
    def transform(self, X, y=None):
        X['text'] = X.apply(self.clean, axis=1)
        return X.values

In [21]:
x_train

,text,Division Name_General,Division Name_General Petite,Division Name_Initmates,Department Name_Bottoms,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Blouses,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
16719,absolutely stunning get right size intricat...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16614,great somebody less jelly belly either rouchin...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
146,pretty bit long bigger read reviewers found ...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
18021,eh s ok thin feel like might say stretch c...,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18866,love top perfect small chested actually ...,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20700,large lace kinda coarse thought d love top ...,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5207,transitional bought dress black think great pi...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18080,fun love bows dress cute flattering comfor...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11528,great addition coat closet ca n t wait get col...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
